# Simple Model Evaluation

Basic evaluation comparing fine-tuned vs base model performance on quote generation.

In [2]:
import mlx_lm
import pandas as pd
import matplotlib.pyplot as plt
from mlx_lm.sample_utils import make_sampler
import json

print("Loading models...")

Loading models...


In [3]:
# Load fine-tuned model
ft_model, ft_tokenizer = mlx_lm.load(
    'mlx-community/Llama-3.2-3B-Instruct-4bit',
    adapter_path='../models/llama3.2-3b-quotes-lora-mlx'
)
print("✅ Fine-tuned model loaded")

# Load base model
base_model, base_tokenizer = mlx_lm.load('mlx-community/Llama-3.2-3B-Instruct-4bit')
print("✅ Base model loaded")

Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 105738.76it/s]


✅ Fine-tuned model loaded


Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 103563.06it/s]


✅ Base model loaded


In [4]:
# Test prompts covering different themes
test_prompts = [
    "Give me advice about perseverance",
    "Give me advice about courage", 
    "Give me advice about success",
    "Give me advice about self-discipline",
    "Give me advice about leadership",
    "Give me advice about personal growth",
    "Give me advice about confidence",
    "Give me advice about motivation"
]

In [5]:
def generate_response(model, tokenizer, prompt, max_tokens=100, temperature=0.7):
    """Generate response from model"""
    sampler = make_sampler(temp=temperature)
    response = mlx_lm.generate(
        model, tokenizer,
        prompt=prompt,
        max_tokens=max_tokens,
        sampler=sampler
    )
    # Clean response by removing the prompt
    if prompt in response:
        response = response.replace(prompt, "").strip()
    return response

In [6]:
# Generate responses for all test prompts
results = []

for prompt in test_prompts:
    print(f"Testing: {prompt}")
    
    # Generate from both models
    ft_response = generate_response(ft_model, ft_tokenizer, prompt)
    base_response = generate_response(base_model, base_tokenizer, prompt)
    
    results.append({
        'prompt': prompt,
        'fine_tuned': ft_response,
        'base_model': base_response,
        'ft_length': len(ft_response.split()),
        'base_length': len(base_response.split())
    })

print(f"\n✅ Generated {len(results)} response pairs")

Testing: Give me advice about perseverance
Testing: Give me advice about courage
Testing: Give me advice about success
Testing: Give me advice about self-discipline
Testing: Give me advice about leadership
Testing: Give me advice about personal growth
Testing: Give me advice about confidence
Testing: Give me advice about motivation

✅ Generated 8 response pairs


In [7]:
# Display results in a nice table format
df = pd.DataFrame(results)
print("📊 Results Summary:\n")

for i, row in df.iterrows():
    print(f"🎯 PROMPT: {row['prompt']}")
    print(f"📚 FINE-TUNED: {row['fine_tuned']}")
    print(f"🔧 BASE MODEL: {row['base_model']}")
    print(f"📏 Lengths: FT={row['ft_length']} words, Base={row['base_length']} words")
    print("-" * 80)
    print()

📊 Results Summary:

🎯 PROMPT: Give me advice about perseverance
📚 FINE-TUNED: 
🔧 BASE MODEL: and motivation. I've been struggling with these aspects of life for a while now.

**Perseverance**

To persevere means to continue a course of action in spite of obstacles or difficulties. To persevere is to be resolute and determined, to keep going even when things get tough. But it's not always easy. Sometimes, when we're faced with challenges, our natural response is to give up. We might feel overwhelmed, frustrated, or defeated.

Here are some tips to help
📏 Lengths: FT=0 words, Base=77 words
--------------------------------------------------------------------------------

🎯 PROMPT: Give me advice about courage
📚 FINE-TUNED: 
Share your wisdom on the power of courage
Take risks
Experience the thrill of risk
Success is not guaranteed
Take a leap of faith
Be the change you want to see
Live in the present
Be a light in the darkness
Be a pioneer
Be a forward thinker
Be courageous in the face of

In [ ]:
# Simple metrics visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Response length comparison
ax1.bar(['Fine-tuned', 'Base'], [df['ft_length'].mean(), df['base_length'].mean()])
ax1.set_title('Average Response Length (words)')
ax1.set_ylabel('Words')

# Length distribution
ax2.hist(df['ft_length'], alpha=0.7, label='Fine-tuned', bins=5)
ax2.hist(df['base_length'], alpha=0.7, label='Base', bins=5)
ax2.set_title('Response Length Distribution')
ax2.set_xlabel('Words')
ax2.set_ylabel('Frequency')
ax2.legend()

plt.tight_layout()
plt.show()

In [8]:
# Basic quality metrics
print("📈 Basic Metrics:")
print(f"Average Fine-tuned Response Length: {df['ft_length'].mean():.1f} words")
print(f"Average Base Model Response Length: {df['base_length'].mean():.1f} words")
print(f"Fine-tuned Length Std: {df['ft_length'].std():.1f}")
print(f"Base Model Length Std: {df['base_length'].std():.1f}")

# Check for quote-like responses (basic heuristic)
ft_quote_like = sum(1 for resp in df['fine_tuned'] if len(resp.split()) < 30 and '.' in resp)
base_quote_like = sum(1 for resp in df['base_model'] if len(resp.split()) < 30 and '.' in resp)

print(f"\nQuote-like responses (< 30 words, contains period):")
print(f"Fine-tuned: {ft_quote_like}/{len(df)} ({ft_quote_like/len(df)*100:.1f}%)")
print(f"Base model: {base_quote_like}/{len(df)} ({base_quote_like/len(df)*100:.1f}%)")

📈 Basic Metrics:
Average Fine-tuned Response Length: 29.1 words
Average Base Model Response Length: 77.5 words
Fine-tuned Length Std: 34.1
Base Model Length Std: 4.8

Quote-like responses (< 30 words, contains period):
Fine-tuned: 1/8 (12.5%)
Base model: 0/8 (0.0%)


In [9]:
# Save results for future reference
with open('../data/evaluation_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("💾 Results saved to ../data/evaluation_results.json")

💾 Results saved to ../data/evaluation_results.json
